In [1]:
# I created the CSV from:
# https://www.transfermarkt.com/zinedine-zidane/elfmetertore/spieler/3111/plus/0?saison_id=ges&wettbewerb_id=

#POINTS TO NOTE:
#the csv is highly imbalanced;Ronaldo has taken more penalties than zidane and it has no missing values.
#both palyers have played on diffrent seasons hence diffrent seasons;therefore season and date wont give better analysis

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #for plotting the data 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from subprocess import check_output

In [2]:
df = pd.read_csv('ronaldo_zidane.csv')
df.head()


,season,competition,date,player,penalyt_scored
0,2019/2000,SerieA,10-30-19,ronaldo,1
1,2018/2019,EuropeanQualifiers,10-14-19,ronaldo,1
2,2019/2000,SerieA,09-21-19,ronaldo,1
3,2018/2019,EuropeanQualifiers,09-10-19,ronaldo,1
4,2018/2019,UEFAChampionsLeague,03-12-19,ronaldo,1


In [3]:
df.describe()


,penalyt_scored
count,42.000000
mean,0.595238
std,0.496796
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
season            42 non-null object
competition       42 non-null object
date              42 non-null object
player            42 non-null object
penalyt_scored    42 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.8+ KB


deal with categorical data

In [5]:
data_set = df[['competition','player']]
one_hot_encoded_training_predictors = pd.get_dummies(data_set)
one_hot_encoded_training_predictors.head()
X = one_hot_encoded_training_predictors
y = df['penalyt_scored']

In [6]:
one_hot_encoded_training_predictors.head()

,competition_EURO,competition_EURO,competition_EuropeanQualifiers,competition_International Friendlies,competition_InternationalFriendlies,competition_LaLiga,competition_SerieA,competition_UEFAChampionsLeague,competition_World Cup 2006,competition_WorldCup,competition_WorldCupQualificationEurope,player_ronaldo,player_zidane
0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,0,1,0


Model training and Evaluation

Logistic Regression and Model Validation

In [7]:
# Defining the predictor and target variables for classification
# in this case we take laliga against the two players
features = ["competition_LaLiga","player_ronaldo","player_zidane"]
X = one_hot_encoded_training_predictors[features]
y = df["penalyt_scored"]


In [8]:
#dividing the data in training and test data

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                     test_size=0.3,
                                                     stratify=y,
                                                     random_state=42)

In [9]:
logreg = LogisticRegression()                               #logistic regression using python
logreg.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
y_pred = logreg.predict(X_test) #predicting the values
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.77


In [11]:
# Testing the Model by introducing a Test or Validation set
scores = cross_val_score(logreg, X_test, y_test, cv=5)
scores

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

array([0.66666667, 0.66666667, 0.66666667, 0.5       , 0.5       ])

In [12]:
# Predict the class probabilities for the Test set
test_prob = logreg.predict_proba(X_test)
test_prob[0:3]

array([[0.39457383, 0.60542617],
       [0.39457383, 0.60542617],
       [0.39457383, 0.60542617]])

In [13]:
 scores.mean() # print "The mean accuracy of 5-fold cross validation on the test set is",

0.6

In [14]:
 scores.std() # print "with a standard deviation of 

0.08164965809277258

In [15]:
logreg.score(X_train, y_train) #print The training set accuracy 

0.6896551724137931

The model does slightly well, with increase of 9 in accuracy score in the test set from the training set in the model,these poor prediction may be due to class imbalance and not having enough features to do the prediction 

USING RANDOM FOREST CLASSFIER

In [21]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import accuracy_score
rfc_cfa = RFC()
rfc_cfa.fit(X_train,y_train)
y_pred = rfc_cfa.predict(X_train)
print('Train accuracy score:',accuracy_score(y_train,y_pred))
print('Test accuracy score:', accuracy_score(y_test,rfc_cfa.predict(X_test)))

Train accuracy score: 0.6896551724137931
Test accuracy score: 0.7692307692307693


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


BOTH MODELS ARE PERFORMING equally same.

CREATING THE CSV FOR PREDICTION

In [23]:
def cfa_model_for_prediction(filename, predictions):
 predict = pd.DataFrame({'playerId':testdf['playerId'],'penalty_scored':predictions})
 submission.to_csv(filename,index=False)
cfa_model_for_prediction("ronaldo_zidane_tree.csv")

TypeError: cfa_model_for_prediction() missing 1 required positional argument: 'predictions'